# Pricing Chart 

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
df = pd.read_csv('C://Users//jack//Documents//price_prediction_pipeline//dataset//base//base.csv')

#display(df)


In [3]:
# List columns
columns = df.columns
#print(columns)


# Drop columns
df = df.drop(columns=['processor speed', 
       'screen size', 'type', 'series',
       'condition', 'processor', 'features', 'seller notes', 'title', 'link',
        'item number'])

display(df)

,price,brand,processor i series,processor generation,ram size,storage capacity,storage type,gpu,operating system,model,sold_date
0,234.99,dell,i7,4,16,256,ssd,nvidia quadro k2100m,windows 11 pro,precision m4800,2023-10-30
1,219.99,dell,i7,4,16,256,ssd,nvidia quadro k2100m,windows 11 pro,precision m4800,2023-10-29
2,249.99,dell,i7,6,16,512,ssd,nvidia quadro m620,windows 11 home,precision 3520,2023-10-26
3,299.99,dell,i7,4,16,512,ssd,nvidia quadro k3100m,windows 11 pro,precision m6600,2023-10-11
4,329.99,dell,i7,9,16,256,nvme,nvidia quadro t1000,not included,precision 5540,2023-10-20
...,...,...,...,...,...,...,...,...,...,...,...
8054,249.99,hp,i5,10,8,256,ssd,integrated,windows 11 home,probook 450 g7,2024-03-20
8055,56.00,intel,i5,4,8,240,ssd,integrated,windows 10,probook 430 g3,2024-03-01
8056,341.99,dell,i5,11,16,512,ssd,integrated,windows 11 pro,inspiron 15 5510,2024-03-19
8057,276.25,dell,i5,11,8,256,ssd,integrated,windows 11 home,inspiron 15 5510,2024-02-13


In [4]:
# Define the dictionary
ram_prices = {4:5, 8:10, 16:20, 32:40}

# Convert the 'ram size' column to int for matching with the dictionary keys
df['ram size'] = df['ram size'].astype(int)

# Create a new column 'ram price' by mapping the 'ram size' column to the ram_prices dictionary
df['ram price'] = df['ram size'].map(ram_prices)

display(df)

,price,brand,processor i series,processor generation,ram size,storage capacity,storage type,gpu,operating system,model,sold_date,ram price
0,234.99,dell,i7,4,16,256,ssd,nvidia quadro k2100m,windows 11 pro,precision m4800,2023-10-30,20.0
1,219.99,dell,i7,4,16,256,ssd,nvidia quadro k2100m,windows 11 pro,precision m4800,2023-10-29,20.0
2,249.99,dell,i7,6,16,512,ssd,nvidia quadro m620,windows 11 home,precision 3520,2023-10-26,20.0
3,299.99,dell,i7,4,16,512,ssd,nvidia quadro k3100m,windows 11 pro,precision m6600,2023-10-11,20.0
4,329.99,dell,i7,9,16,256,nvme,nvidia quadro t1000,not included,precision 5540,2023-10-20,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8054,249.99,hp,i5,10,8,256,ssd,integrated,windows 11 home,probook 450 g7,2024-03-20,10.0
8055,56.00,intel,i5,4,8,240,ssd,integrated,windows 10,probook 430 g3,2024-03-01,10.0
8056,341.99,dell,i5,11,16,512,ssd,integrated,windows 11 pro,inspiron 15 5510,2024-03-19,20.0
8057,276.25,dell,i5,11,8,256,ssd,integrated,windows 11 home,inspiron 15 5510,2024-02-13,10.0


In [5]:
import numpy as np
from sklearn.linear_model import LinearRegression
import re

# Define the dictionary
storage_prices = {
    '500 hdd': 10.00,
    '1000 hdd': 20.00,
    '256 ssd': 20.00,
    '128 ssd': 10.00,
    '512 ssd': 40.00,
    '1000 ssd': 80.00
}

type_dict = {
    'ssd': 'ssd',
    'nvme': 'ssd',
    'sshd': 'hdd',
    'hdd': 'hdd',
    'm.2': 'ssd',
    'emmc': 'ssd',
    'solid state drive': 'ssd',
    'm2': 'ssd',
    'm2 chip': 'ssd',
    'msata': 'ssd'
}

# Convert storage type entries by dictionary
df['storage type'] = df['storage type'].str.lower().map(type_dict)

# Extract the numeric sizes from the keys of the storage_prices dictionary
sizes = np.array([re.search(r'\d+', key).group() for key in storage_prices.keys()]).astype(float).reshape(-1, 1)

# The rest of the code remains the same
prices = np.array(list(storage_prices.values()))

# Take the logarithm of the sizes and prices
log_sizes = np.log(sizes)
log_prices = np.log(prices)

# Fit a linear model to the log-sizes and log-prices
model = LinearRegression().fit(log_sizes, log_prices)

# Predict the log-prices for the missing sizes
missing_sizes = np.array([256, 512, 180, 240, 128, 120, 480, 500, 250, 750, 275, 798, 200, 167, 64, 768, 953, 700, 192, 454, 237, 1024, 400, 960, 160, 320, 4096, 230, 297]).reshape(-1, 1)
log_predicted_prices = model.predict(np.log(missing_sizes))

# Convert the predicted log-prices back to prices
predicted_prices = np.exp(log_predicted_prices)

# Create a dictionary of the missing sizes and their predicted prices
predicted_prices_dict = dict(zip(missing_sizes.flatten(), predicted_prices.flatten()))

# Update the storage_prices dictionary with the predicted_prices_dict
storage_prices.update(predicted_prices_dict)

# Combine the 'storage capacity' and 'storage type' columns
df['storage'] = df['storage capacity'].astype(str) + ' ' + df['storage type'].astype(str)

# Ensure that the 'storage' column is formatted consistently with the keys in the storage_prices dictionary
df['storage'] = df['storage'].str.lower().str.strip()

# Replace the values in the 'storage' column with the corresponding values from the dictionary
df['storage price'] = df['storage'].map(storage_prices)

# Fill NaN values with the median of the non-NaN values in the 'storage price' column
df['storage price'] = df['storage price'].fillna(df['storage price'].median())

# The rest of the code remains the same
df[['storage capacity', 'storage type']] = df['storage'].str.split(' ', n=1, expand=True)
df = df.drop(columns=['storage'])

display(df)

,price,brand,processor i series,processor generation,ram size,storage capacity,storage type,gpu,operating system,model,sold_date,ram price,storage price
0,234.99,dell,i7,4,16,256,ssd,nvidia quadro k2100m,windows 11 pro,precision m4800,2023-10-30,20.0,20.0
1,219.99,dell,i7,4,16,256,ssd,nvidia quadro k2100m,windows 11 pro,precision m4800,2023-10-29,20.0,20.0
2,249.99,dell,i7,6,16,512,ssd,nvidia quadro m620,windows 11 home,precision 3520,2023-10-26,20.0,40.0
3,299.99,dell,i7,4,16,512,ssd,nvidia quadro k3100m,windows 11 pro,precision m6600,2023-10-11,20.0,40.0
4,329.99,dell,i7,9,16,256,ssd,nvidia quadro t1000,not included,precision 5540,2023-10-20,20.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8054,249.99,hp,i5,10,8,256,ssd,integrated,windows 11 home,probook 450 g7,2024-03-20,10.0,20.0
8055,56.00,intel,i5,4,8,240,ssd,integrated,windows 10,probook 430 g3,2024-03-01,10.0,20.0
8056,341.99,dell,i5,11,16,512,ssd,integrated,windows 11 pro,inspiron 15 5510,2024-03-19,20.0,40.0
8057,276.25,dell,i5,11,8,256,ssd,integrated,windows 11 home,inspiron 15 5510,2024-02-13,10.0,20.0


In [6]:
# Add a new column called base_price, which is price minus the sum of the ram price and storage price
df['base_price'] = df['price'] - df['ram price'] - df['storage price']

display(df)

,price,brand,processor i series,processor generation,ram size,storage capacity,storage type,gpu,operating system,model,sold_date,ram price,storage price,base_price
0,234.99,dell,i7,4,16,256,ssd,nvidia quadro k2100m,windows 11 pro,precision m4800,2023-10-30,20.0,20.0,194.99
1,219.99,dell,i7,4,16,256,ssd,nvidia quadro k2100m,windows 11 pro,precision m4800,2023-10-29,20.0,20.0,179.99
2,249.99,dell,i7,6,16,512,ssd,nvidia quadro m620,windows 11 home,precision 3520,2023-10-26,20.0,40.0,189.99
3,299.99,dell,i7,4,16,512,ssd,nvidia quadro k3100m,windows 11 pro,precision m6600,2023-10-11,20.0,40.0,239.99
4,329.99,dell,i7,9,16,256,ssd,nvidia quadro t1000,not included,precision 5540,2023-10-20,20.0,20.0,289.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8054,249.99,hp,i5,10,8,256,ssd,integrated,windows 11 home,probook 450 g7,2024-03-20,10.0,20.0,219.99
8055,56.00,intel,i5,4,8,240,ssd,integrated,windows 10,probook 430 g3,2024-03-01,10.0,20.0,26.00
8056,341.99,dell,i5,11,16,512,ssd,integrated,windows 11 pro,inspiron 15 5510,2024-03-19,20.0,40.0,281.99
8057,276.25,dell,i5,11,8,256,ssd,integrated,windows 11 home,inspiron 15 5510,2024-02-13,10.0,20.0,246.25


In [7]:
# Average base price calculation

# Assuming df is your dataframe
df['base_price'] = df['base_price'].astype(float)  # Ensure base_price is float for averaging

# Group by and calculate average base_price and count, then round average base_price to 2 decimal places
grouped_df = df.groupby(['brand', 'model', 'processor generation', 'processor i series']).agg({'base_price': ['mean', 'count']})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
grouped_df['base_price_mean'] = grouped_df['base_price_mean'].round(2)

# Unstack processor i series
avg_base_price_df = grouped_df['base_price_mean'].unstack().reset_index()
count_df = grouped_df['base_price_count'].unstack().reset_index()

# Get latest sold_date
latest_sold_date_df = df.groupby(['brand', 'model', 'processor generation'])['sold_date'].max().reset_index()

# Merge dataframes
final_df = pd.merge(avg_base_price_df, latest_sold_date_df, on=['brand', 'model', 'processor generation'])
final_df = pd.merge(final_df, count_df, on=['brand', 'model', 'processor generation'], suffixes=('', '_count'))

# Filter for brands dell, hp , lenovo
#'final_df = final_df[final_df['brand'].isin(['dell', 'hp', 'lenovo'])]

# save to csv
final_df.to_csv('C://Users//jack//Documents//price_prediction_pipeline//dataset//pricing//average_base_prices.csv', index=False)

# Display final_df
display(final_df)

,brand,model,processor generation,i3,i5,i7,i9,sold_date,i3_count,i5_count,i7_count,i9_count
0,acer,15-da0xxx,11,160.0,NaN,NaN,NaN,2023-08-15,1.0,NaN,NaN,NaN
1,acer,17-by2xxx,6,NaN,NaN,170.00,NaN,2023-08-31,NaN,NaN,1.0,NaN
2,acer,aspire,4,NaN,34.99,NaN,NaN,2024-02-14,NaN,2.0,NaN,NaN
3,acer,aspire 5336,4,NaN,109.99,NaN,NaN,2023-09-11,NaN,1.0,NaN,NaN
4,acer,aspire 5336,7,NaN,NaN,199.10,NaN,2023-10-25,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1235,toshiba,tecra z40-c,4,NaN,64.99,NaN,NaN,2023-08-29,NaN,1.0,NaN,NaN
1236,toshiba,tecra z40-c,5,NaN,NaN,89.99,NaN,2023-09-06,NaN,NaN,1.0,NaN
1237,toshiba,tecra z40-c,6,NaN,39.95,NaN,NaN,2024-02-15,NaN,6.0,NaN,NaN
1238,toshiba,tecra z40-c,7,NaN,53.99,NaN,NaN,2023-12-15,NaN,1.0,NaN,NaN


In [8]:
# Max price calculation

# Assuming df is your dataframe
df['base_price'] = df['base_price'].astype(float)  # Ensure base_price is float for max calculation

# Group by and calculate max base_price and count
grouped_df = df.groupby(['brand', 'model', 'processor generation', 'processor i series']).agg({'base_price': ['max', 'count']})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]

# Unstack processor i series
max_base_price_df = grouped_df['base_price_max'].unstack().reset_index()
count_df = grouped_df['base_price_count'].unstack().reset_index()

# Get latest sold_date
latest_sold_date_df = df.groupby(['brand', 'model', 'processor generation'])['sold_date'].max().reset_index()

# Merge dataframes
final_df = pd.merge(max_base_price_df, latest_sold_date_df, on=['brand', 'model', 'processor generation'])
final_df = pd.merge(final_df, count_df, on=['brand', 'model', 'processor generation'], suffixes=('', '_count'))

# Filter for brands dell, hp , lenovo
# final_df = final_df[final_df['brand'].isin(['dell', 'hp', 'lenovo'])]

# save to csv
final_df.to_csv('C://Users//jack//Documents//price_prediction_pipeline//dataset//pricing//max_base_prices.csv', index=False)

# Display final_df
display(final_df)

,brand,model,processor generation,i3,i5,i7,i9,sold_date,i3_count,i5_count,i7_count,i9_count
0,acer,15-da0xxx,11,160.0,NaN,NaN,NaN,2023-08-15,1.0,NaN,NaN,NaN
1,acer,17-by2xxx,6,NaN,NaN,170.00,NaN,2023-08-31,NaN,NaN,1.0,NaN
2,acer,aspire,4,NaN,39.99,NaN,NaN,2024-02-14,NaN,2.0,NaN,NaN
3,acer,aspire 5336,4,NaN,109.99,NaN,NaN,2023-09-11,NaN,1.0,NaN,NaN
4,acer,aspire 5336,7,NaN,NaN,199.10,NaN,2023-10-25,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1235,toshiba,tecra z40-c,4,NaN,64.99,NaN,NaN,2023-08-29,NaN,1.0,NaN,NaN
1236,toshiba,tecra z40-c,5,NaN,NaN,89.99,NaN,2023-09-06,NaN,NaN,1.0,NaN
1237,toshiba,tecra z40-c,6,NaN,55.00,NaN,NaN,2024-02-15,NaN,6.0,NaN,NaN
1238,toshiba,tecra z40-c,7,NaN,53.99,NaN,NaN,2023-12-15,NaN,1.0,NaN,NaN


In [9]:
# Min price calculation

# Assuming df is your dataframe
df['base_price'] = df['base_price'].astype(float)  # Ensure base_price is float for min calculation

# Group by and calculate min base_price and count
grouped_df = df.groupby(['brand', 'model', 'processor generation', 'processor i series']).agg({'base_price': ['min', 'count']})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]

# Unstack processor i series
min_base_price_df = grouped_df['base_price_min'].unstack().reset_index()
count_df = grouped_df['base_price_count'].unstack().reset_index()

# Get latest sold_date
latest_sold_date_df = df.groupby(['brand', 'model', 'processor generation'])['sold_date'].max().reset_index()

# Merge dataframes
final_df = pd.merge(min_base_price_df, latest_sold_date_df, on=['brand', 'model', 'processor generation'])
final_df = pd.merge(final_df, count_df, on=['brand', 'model', 'processor generation'], suffixes=('', '_count'))

# Filter for brands dell, hp , lenovo
# final_df = final_df[final_df['brand'].isin(['dell', 'hp', 'lenovo'])]

# save to csv
final_df.to_csv('C://Users//jack//Documents//price_prediction_pipeline//dataset//pricing//min_base_prices.csv', index=False)

# Display final_df
display(final_df)

,brand,model,processor generation,i3,i5,i7,i9,sold_date,i3_count,i5_count,i7_count,i9_count
0,acer,15-da0xxx,11,160.0,NaN,NaN,NaN,2023-08-15,1.0,NaN,NaN,NaN
1,acer,17-by2xxx,6,NaN,NaN,170.00,NaN,2023-08-31,NaN,NaN,1.0,NaN
2,acer,aspire,4,NaN,30.00,NaN,NaN,2024-02-14,NaN,2.0,NaN,NaN
3,acer,aspire 5336,4,NaN,109.99,NaN,NaN,2023-09-11,NaN,1.0,NaN,NaN
4,acer,aspire 5336,7,NaN,NaN,199.10,NaN,2023-10-25,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1235,toshiba,tecra z40-c,4,NaN,64.99,NaN,NaN,2023-08-29,NaN,1.0,NaN,NaN
1236,toshiba,tecra z40-c,5,NaN,NaN,89.99,NaN,2023-09-06,NaN,NaN,1.0,NaN
1237,toshiba,tecra z40-c,6,NaN,14.98,NaN,NaN,2024-02-15,NaN,6.0,NaN,NaN
1238,toshiba,tecra z40-c,7,NaN,53.99,NaN,NaN,2023-12-15,NaN,1.0,NaN,NaN


In [10]:
# Based on brand and model count how many of each thera and store the results in a dictionary as brand+model as the key and the count as the value

# Assuming your DataFrame is named 'df'
counts = df.groupby(['brand', 'model']).size().to_dict()

# The keys in the dictionary will be tuples of the form ('brand', 'model')

df['brand_model'] = df['brand'] + ' ' + df['model']
counts = df.groupby('brand_model').size().to_dict()


# Reorder the dictionary by value
sorted_counts = {k: v for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True)}

# Display the sorted counts
for key, value in sorted_counts.items():
    print(key, value)

hp probook 445r g6 202
hp elitebook 830 g5 114
dell system vostro 3450 105
hp elitebook 840 g4 101
dell latitude 7490 93
dell latitude e5470 91
dell latitude 7390 89
dell latitude 7280 88
dell latitude 7480 86
dell xps 13 9370 86
dell latitude 5420 80
hp elitebook 820 g3 79
lenovo thinkpad l380 77
dell latitude 5590 76
lenovo thinkpad x270 75
dell latitude e7470 74
dell xps 13 9380 74
dell latitude 5400 73
dell latitude 5410 73
hp elitebook 840 g6 73
dell latitude e5570 72
dell xps 13 9360 72
dell latitude e7270 71
dell latitude 5480 70
lenovo thinkpad e14 70
dell xps 13 7390 69
lenovo thinkpad t440p 65
dell latitude 5580 63
dell latitude e7450 62
lenovo thinkpad t470 61
lenovo thinkpad x260 60
dell latitude 5490 58
dell latitude 7290 58
hp elitebook 840 g7 58
lenovo thinkpad t480s 58
hp elitebook 840 g3 57
hp 250 g7 56
lenovo thinkpad t460 56
hp elitebook 840 g5 55
hp probook 450 g3 53
dell latitude 7400 52
dell xps 15 9560 50
lenovo thinkpad x240 50
dell latitude e7440 48
dell precis